We will be working with three bonds:
* B1 is a 15 Year Bond with a Face Value of USD 1000 that pays a 5 percent coupon semi-annually (2 times a year)
* B2 is a 5 Year Bond with a Face value of USD 1000 that pays a 6 percent coupon quarterly (4 times a year)
* B3 is a 10 Year Zero-Coupon Bond with a Face Value of USD 1000 

-- Hint: you can still use the ash.bond_cash_flows() and ash.bond_price() by setting the coupon amount to 0 percent and the coupons_per_year to 1)
Assume the yield curve is flat at 5 percent.

In [1]:
import ashmodule as ash
import pandas as pd
import numpy as np
import matplotlib as plt

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
b1_price = ash.bond_price(15,principal=1000,coupon_rate=0.05,coupons_per_year=2,discount_rate=0.05)
b2_price = ash.bond_price(5,principal=1000,coupon_rate=0.06,coupons_per_year=4,discount_rate=0.05)
b3_price = ash.bond_price(10,principal=1000,coupon_rate=0,coupons_per_year=1,discount_rate=0.05)
print(b1_price,"\n",b2_price,"\n",b3_price)

0    1000.0
dtype: float64 
 0    1043.99829
dtype: float64 
 0    613.913254
dtype: float64


In [3]:
b1_cf = ash.bond_cash_flows(15,principal=1000,coupon_rate=0.05,coupons_per_year=2)
b2_cf = ash.bond_cash_flows(5,principal=1000,coupon_rate=0.06,coupons_per_year=4)
b3_cf = ash.bond_cash_flows(10,principal=1000,coupon_rate=0,coupons_per_year=1)

In [19]:
import erk
b1_md = erk.macaulay_duration(b1_cf,0.05)
b2_md = erk.macaulay_duration(b2_cf,0.05)
b3_md = erk.macaulay_duration(b3_cf,0.05)
print(b1_md,"\n",b2_md,"\n",b3_md)

18.74518415380995 
 16.32082218369695 
 10.0


In [11]:
liabilities = pd.Series(data = [100000,200000,300000],index = [3,5,10])
liabilities

3     100000
5     200000
10    300000
dtype: int64

In [16]:
liab_md = erk.macaulay_duration(liabilities,0.05)
liab_md

6.750917852744651

In [20]:
(b2_md - liab_md)/(b2_md - b1_md)

-3.9473908801276245

In [21]:
(b3_md - liab_md)/(b3_md - b2_md)

-0.514028405297586

In [46]:
def macaulay_duration(flows, discount_rate,coupons_per_year = 1):
    """
    Computes the Macaulay Duration of a sequence of cash flows, given a per-period discount rate
    """
    discount_rate = discount_rate/coupons_per_year
    discounted_flows = erk.discount(flows.index, discount_rate)*pd.DataFrame(flows)
    weights = discounted_flows/discounted_flows.sum()
    return np.average(flows.index, weights=weights.iloc[:,0])

def match_durations(cf_liab, cf_short,cf_long,discount_rate, short_c_per_year = 2, long_c_per_year = 4):
    """
    Returns the weight W in cf_s that, along with (1-W) in cf_l will have an effective
    duration that matches cf_t
    """
    d_liab = macaulay_duration(cf_liab, discount_rate)
    d_short = macaulay_duration(cf_short, discount_rate,coupons_per_year=short_c_per_year)
    d_long =  macaulay_duration(cf_long, discount_rate,coupons_per_year= long_c_per_year)
    return (d_long - d_liab)/(d_long - d_short)

match_durations(liabilities,cf_short=b1_cf,cf_long=b2_cf,discount_rate=0.05)


-2.7126949141880514

In [35]:
(-0.514028405297586*b2_md)+((1-0.514028405297586)*b3_md)

-3.529650253207068

In [47]:
match_durations(liabilities,cf_short=b2_cf,cf_long=b3_cf,discount_rate=0.05)

-0.45494144499856126

In [67]:
def macaulay_duration(flows, discount_rate,coupons_per_year = 1):
    """
    Computes the Macaulay Duration of a sequence of cash flows, given a per-period discount rate
    """
    discount_rate = discount_rate/coupons_per_year
    
    discounted_flows = erk.discount(flows.index, discount_rate)*pd.DataFrame(flows)
    weights = discounted_flows/discounted_flows.sum()
    return (np.average(flows.index, weights=weights.iloc[:,0]))/coupons_per_year

In [68]:

b1_md = macaulay_duration(b1_cf,0.05,coupons_per_year=2)
b2_md = macaulay_duration(b2_cf,0.05,coupons_per_year=4)
b3_md = macaulay_duration(b3_cf,0.05,coupons_per_year=1)
print(b1_md,"\n",b2_md,"\n",b3_md)

10.72677495379012 
 4.373363222636413 
 10.0


In [69]:
def match_durations(cf_liab, cf_short,cf_long,discount_rate, short_c_per_year = 4, long_c_per_year = 2):
    """
    Returns the weight W in cf_s that, along with (1-W) in cf_l will have an effective
    duration that matches cf_t
    """
    d_liab = macaulay_duration(cf_liab, discount_rate)
    d_short = macaulay_duration(cf_short, discount_rate,coupons_per_year=short_c_per_year)
    d_long =  macaulay_duration(cf_long, discount_rate,coupons_per_year= long_c_per_year)
    return (d_long - d_liab)/(d_long - d_short)

In [83]:
por_2for1 = match_durations(liabilities,cf_short=b2_cf,cf_long=b1_cf,discount_rate=0.05,short_c_per_year = 4, long_c_per_year = 2)
round(por_2for1*100,2)

62.58

In [78]:
por_1for2 = 1-por_2for1
por_1for2

0.37421699249396856

In [79]:
por_2for1*b2_md + por_1for2*b1_md == liab_md

True

In [81]:
round(match_durations(liabilities,cf_short=b2_cf,cf_long=b3_cf,discount_rate=0.05,short_c_per_year = 4, long_c_per_year = 1)*100,2)

57.74